In [57]:
import pandas as pd
from collections import namedtuple, Counter
import copy

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src/')
from recourse import optimization
from recourse import get_logistic_param

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import lightgbm as lgb
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

## Model

In [58]:
model = LGBMClassifier(objective = 'binary', predict_raw_score = True)

feature_dir = "../data/out/airbnb_features.csv"
target_dir = "../data/out/airbnb_target.csv"
X = pd.read_csv(feature_dir, index_col = 0)
columns = X.columns
X.columns = range(X.shape[1])
y = pd.read_csv(target_dir, index_col = 0)

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 42)
model = model.fit(train_X, train_y)

C:\Users\14388\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\14388\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [59]:
train_X.columns = columns
test_X.columns = columns

In [60]:
pred_test_y = model.predict(test_X)
accuracy_score(pred_test_y, test_y)

0.8579465541490858

In [61]:
df = test_X.copy()
df['test_y'] = test_y
df = df.reset_index(drop = True)
df['pred_test_y'] = pd.Series(pred_test_y)

In [62]:
df

,host_response_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,security_deposit,cleaning_fee,guests_included,extra_people,...,instant_bookable: t,cancellation_policy: flexible,cancellation_policy: moderate,cancellation_policy: strict,require_guest_profile_picture: f,require_guest_profile_picture: t,require_guest_phone_verification: f,require_guest_phone_verification: t,test_y,pred_test_y
0,100.0,2.0,2.0,1.5,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0
1,100.0,1.0,2.0,1.0,1.0,1.0,0.0,30.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,0
2,88.0,36.0,6.0,2.0,2.0,4.0,0.0,209.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1,1
3,100.0,3.0,4.0,1.0,1.0,2.0,0.0,169.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1,1
4,100.0,9.0,6.0,2.0,1.0,4.0,250.0,135.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,100.0,2.0,2.0,1.0,1.0,1.0,100.0,10.0,2.0,15.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,0
707,69.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,1.0,10.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,0
708,100.0,2.0,4.0,1.0,2.0,3.0,200.0,25.0,4.0,50.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,1
709,100.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,20.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,0


## actionable attributes

In [63]:
df.minimum_nights.value_counts()

2.0     279
1.0     276
3.0     101
4.0      22
5.0      15
7.0       6
30.0      3
6.0       2
14.0      2
20.0      2
10.0      2
31.0      1
Name: minimum_nights, dtype: int64

In [64]:
df['guests_included'].value_counts()

1.0     379
2.0     222
4.0      49
3.0      20
0.0      16
6.0      15
8.0       5
7.0       2
5.0       2
10.0      1
Name: guests_included, dtype: int64

## recourses for first individual

In [72]:
df.iloc[2].minimum_nights

3.0

In [73]:
df.iloc[2]['guests_included']

1.0

In [71]:
df.iloc[2]['pred_test_y']

1.0

In [74]:
A=['minimum_nights','guests_included']
aval=[3,1]

klst=df.drop(['minimum_nights','guests_included', 'test_y', 'pred_test_y'], axis = 1).columns.to_list()
kval=df.drop(['minimum_nights','guests_included', 'test_y', 'pred_test_y'], axis = 1).iloc[2].to_list()

conditionals=copy.deepcopy(A)
conditionals.extend(klst)

(beta_lst,[beta0])=get_logistic_param(df,conditionals,['pred_test_y'],[1])

Adomain=[[1,2,3,4,5,6,7,10,14,20,30,31],[1,2,3,4,5,6,7,8,9,10]]#List of list where each list is domain of each variable in A

optimization(df,A,aval,Adomain,klst,kval,0.819,beta_lst,beta0,['pred_test_y'])

C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

0.9992281025395493 0.0031392410452329676
beta list is  [-0.020091671782907043, 0.19251556165929556, -0.0006673563917889118, -0.005644271594153631, 0.413410186973757, 0.08352322724800727, 0.21039874794447058, 0.20319330546422726, 0.0009960829064175679, 0.023916960933903535, -0.022220739241188097, -0.00019236540709539364, -0.01925548345805034, 0.07773092139051112, -0.05443865439762754, 0.0003688791413414581, -0.009067127137892416, -0.12967063499406198, -0.03408259514029189, -0.0021770835370012384, 0.0070313945865444144, 0.009355513666869767, -0.02206073098425469, -0.008041348698309013, 0.00019044243046761894, 0.00191922909819635, 0.0, -0.0002007347897348215, -0.0029206353520579245, 0.00018559235370180314, 0.009288072430065096, -0.005056605568874365, -0.00025572071135415584, -0.0006514225541837934, -0.0003571312019145101, -0.0024967350333484854, 0.010467616213287343, 0.0020849010568587487, 0.010691766382980536, 0.0, -0.004110824443127846, -0.0031924203094473654, -0.00013611128315433807, 0

C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\14388\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modul

In [24]:
import zipfile


In [26]:
feature_dir = "../data/out/loan_downsampled.csv"
loan = pd.read_csv(feature_dir)

In [28]:
loan

,NAME_CONTRACT_TYPE: Cash loans,NAME_CONTRACT_TYPE: Revolving loans,CODE_GENDER: F,CODE_GENDER: M,CODE_GENDER: XNA,FLAG_OWN_CAR: N,FLAG_OWN_CAR: Y,FLAG_OWN_REALTY: N,FLAG_OWN_REALTY: Y,NAME_TYPE_SUITE: Children,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Target
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,1
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1
4,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49645,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
49646,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0
49647,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
49648,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0


In [30]:
model = LGBMClassifier(objective = 'binary', predict_raw_score = True)

X = loan.drop(['Target'], 1)
columns = X.columns
X.columns = range(X.shape[1])
y = loan[['Target']]

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 42)
model = model.fit(train_X, train_y)

train_X.columns = columns
test_X.columns = columns
pred_test_y = model.predict(test_X)
accuracy_score(pred_test_y, test_y)

<ipython-input-30-efbdc5ab7e25>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = loan.drop(['Target'], 1)
C:\Users\14388\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\14388\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6840409248368646

In [31]:
df = test_X.copy()
df['test_y'] = test_y
df = df.reset_index(drop = True)
df['pred_test_y'] = pd.Series(pred_test_y)

In [35]:
df.columns.to_list()

['NAME_CONTRACT_TYPE: Cash loans',
 'NAME_CONTRACT_TYPE: Revolving loans',
 'CODE_GENDER: F',
 'CODE_GENDER: M',
 'CODE_GENDER: XNA',
 'FLAG_OWN_CAR: N',
 'FLAG_OWN_CAR: Y',
 'FLAG_OWN_REALTY: N',
 'FLAG_OWN_REALTY: Y',
 'NAME_TYPE_SUITE: Children',
 'NAME_TYPE_SUITE: Family',
 'NAME_TYPE_SUITE: Group of people',
 'NAME_TYPE_SUITE: Other_A',
 'NAME_TYPE_SUITE: Other_B',
 'NAME_TYPE_SUITE: Spouse, partner',
 'NAME_TYPE_SUITE: Unaccompanied',
 'NAME_TYPE_SUITE: nan',
 'NAME_INCOME_TYPE: Businessman',
 'NAME_INCOME_TYPE: Commercial associate',
 'NAME_INCOME_TYPE: Maternity leave',
 'NAME_INCOME_TYPE: Pensioner',
 'NAME_INCOME_TYPE: State servant',
 'NAME_INCOME_TYPE: Student',
 'NAME_INCOME_TYPE: Unemployed',
 'NAME_INCOME_TYPE: Working',
 'NAME_EDUCATION_TYPE: Academic degree',
 'NAME_EDUCATION_TYPE: Higher education',
 'NAME_EDUCATION_TYPE: Incomplete higher',
 'NAME_EDUCATION_TYPE: Lower secondary',
 'NAME_EDUCATION_TYPE: Secondary / secondary special',
 'NAME_FAMILY_STATUS: Civil marr

In [49]:
df['FLAG_OWN_REALTY: Y'].value_counts()

1.0    8567
0.0    3846
Name: FLAG_OWN_REALTY: Y, dtype: int64

In [52]:
df.loc[1, 'NAME_INCOME_TYPE: Working']

1.0

In [54]:
df.loc[1, 'FLAG_OWN_REALTY: Y']

0.0

In [51]:
df.loc[1, 'pred_test_y']

1

In [ ]:
df.loc[0, 'NAME_INCOME_TYPE: Working']

In [55]:
A=['NAME_INCOME_TYPE: Working','FLAG_OWN_REALTY: Y']
aval=[1,0]
klst=df.drop(['NAME_INCOME_TYPE: Working','FLAG_OWN_REALTY: Y', 'test_y', 'pred_test_y'], axis = 1).columns.to_list()
kval=df.drop(['NAME_INCOME_TYPE: Working','FLAG_OWN_REALTY: Y', 'test_y', 'pred_test_y'], axis = 1).iloc[0].to_list()

conditionals=copy.deepcopy(A)
conditionals.extend(klst)

(beta_lst,[beta0])=get_logistic_param(df,conditionals,['pred_test_y'],[1])

Adomain=[[0,1],[0,1]]#List of list where each list is domain of each variable in A

optimization(df,A,aval,Adomain,klst,kval,0.819,beta_lst,beta0,['pred_test_y'])

C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

0.5695376169352581 0.08615208197315735
beta list is  [1.9868283330015108e-07, 4.2619946784479336e-08, 1.5211211996364743e-07, -4.060615137738738e-08, -1.2663272618843548e-07, 2.3813869477475527e-07, 0.0, 1.5681488344667315e-07, -4.530891486039772e-08, 6.888602180178593e-08, -3.1090335036230422e-09, -8.510593527046813e-09, 6.002608829535068e-10, -3.8128881618496164e-10, 1.8957476884105187e-09, 5.386917938168797e-09, 1.13085927770871e-07, 2.5380301527186203e-09, 0.0, -3.962866358495765e-08, 3.378196901018443e-10, -2.73751642233693e-09, -4.5675563089634576e-08, 0.0, 5.270586929757284e-10, 7.193917069630682e-11, -1.9904144798586085e-07, 3.5252506781715017e-09, 1.29626333722496e-08, 2.939875933510723e-07, 5.1800718701450355e-08, -2.039283826601684e-08, 3.788983690949336e-09, 8.726313061754474e-08, 0.0, -1.0954026157642432e-08, 5.870102463925544e-10, 3.702989727802628e-08, 7.975652900047852e-09, -2.268381570317296e-09, 2.238108311292577e-08, 4.5800706619185267e-08, 3.2487754899583845e-08, 5.

C:\Users\14388\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\14388\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
